Here we make a Singular value Decompoition of the Participant Input-Output Matrix

In [1]:
import numpy as np
import pandas as pd
import re
import os
import glob
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors

get the dir

In [2]:
# the name of our directory
dname = os.path.dirname(os.getcwd()) + "/data_loading/data_v2_quotes/"
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts/data_loading/data_v2_quotes/'

Load our matrices of performance per trial and per block

In [9]:
in_out_matrix = np.load(dname + 'in_out_matrix.npy')
print(in_out_matrix.shape)

in_out_matrix_trialwise = np.load(dname + 'in_out_matrix_trialwise.npy')
print(in_out_matrix_trialwise.shape)


(7, 4, 8)
(7, 4, 128)


Perform the SVD

In [11]:
# u corresponds to left singular vectors, vt to right singular vectors, and s has singular values on the diagonal
u, s, vt = np.linalg.svd(in_out_matrix[:,:,0], full_matrices=True)